In [ ]:
import asyncio
import json
import os
import pathlib

In [ ]:
cwd = pathlib.Path().cwd()
proj_dir = cwd.parent.parent.parent
sample_objects_dir = proj_dir / 'sample_objects'
os.chdir(proj_dir)

from utils.arcgis_helpers import checkout_token
from utils.arcgis_helpers import AsyncArcGISRequester

In [ ]:
nifc_token = checkout_token('NIFC_AGO', 120, 'NIFC_TOKEN', 15)

# save current state of service, so features can be replaced to replicate state.

In [ ]:
async with AsyncArcGISRequester() as requester:
    snapshot_0 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/0',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )
    snapshot_1 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/1',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )
    snapshot_2 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/2',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )
    snapshot_3 = await requester.arcgis_rest_api_get(
        base_url=r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/3',
        params={'f':'json', 'token':nifc_token, 'where':'1=1', 'outfields':'*'},
        operation='query?'
    )

In [ ]:
print(sample_objects_dir)

In [ ]:
for name,response in {
    'perims_locs': snapshot_0,
    'buf_1': snapshot_1,
    'buf_3': snapshot_2,
    'buf_5': snapshot_3
}.items():
    feats = response['features']
    print(len(feats))
    with open(sample_objects_dir / f'{name}.json', 'w') as file:
        json.dump(feats, file)
    


# read service state snapshots and add features to service to replicate state

In [ ]:
feats_dict = {
    'perims_locs': None,
    'buf_1': None,
    'buf_3': None,
    'buf_5': None
}

for name,_ in feats_dict.items():
    with open(sample_objects_dir / f'{name}.json', 'r') as file:
        feats = json.load(file)
    feats_dict[name] = feats

lyr_idx_dict = {
    'perims_locs': 0,
    'buf_1': 1,
    'buf_3': 2,
    'buf_5': 3,
}

async with AsyncArcGISRequester() as requester:
    for name, idx in lyr_idx_dict.items():
        response = await requester.applyEdits_request(
            url=fr'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer/{idx}',
            token=nifc_token,
            features_to_add=feats_dict[name],
            get_oids_to_delete_query='1=1'
        )
        print(response)
